In [ ]:
import sqlite3
import pandas as pd

In [ ]:
df_whr = pd.read_csv('WHR20_Data.csv')

In [ ]:
conn = sqlite3.connect('happiness_database.db')

In [ ]:
table_whr = 'whr_2020'
df_whr.to_sql(table_whr, conn, if_exists='replace', index=False)
conn.close()

In [ ]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_whr}")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

In [ ]:
df_wvs = pd.read_csv('WVS_Wave_7.csv')

In [ ]:
conn = sqlite3.connect('happiness_database.db')
table_wvs = 'wvs_wave7'
df_wvs.to_sql(table_wvs, conn, if_exists='replace', index=False)
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_wvs}")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

In [ ]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
wvs1 = cursor.execute('''
CREATE TABLE wvs1(
    country TEXT Primary Key NOT NULL,
    year INTEGER NOT NULL,
    pm2 INTEGER,
    secval FLOAT,
    skepticism FLOAT,
    relativism FLOAT,
    disbelief FLOAT,
    defiance FLOAT,
    emval FLOAT,
    autonomy FLOAT,
    equality FLOAT,
    choice FLOAT,
    voice FLOAT  
               )
''')
conn.commit()